# Generate Phoneme Dataset from HW1P2

In [1]:
!pip install kaggle
!mkdir .kaggle

In [2]:
import json
token = {"username":"martamendezsimon","key":"6c04747f7b80b8647fec35b5202034a2"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


Download and unzip data from Kaggle competition

In [5]:
!kaggle competitions download -c 11785-spring2021-hw1p2

 92% 130M/142M [00:00<00:00, 118MB/s]
100% 142M/142M [00:01<00:00, 126MB/s]
  0% 0.00/3.43M [00:00<?, ?B/s]
100% 3.43M/3.43M [00:00<00:00, 112MB/s]
 98% 305M/310M [00:08<00:00, 42.1MB/s]
100% 310M/310M [00:08<00:00, 38.4MB/s]
  0% 0.00/518 [00:00<?, ?B/s]
100% 518/518 [00:00<00:00, 527kB/s]
 97% 313M/322M [00:06<00:00, 50.7MB/s]
100% 322M/322M [00:06<00:00, 51.0MB/s]
100% 3.52G/3.52G [01:07<00:00, 38.3MB/s]
100% 3.52G/3.52G [01:07<00:00, 56.2MB/s]
  0% 0.00/500k [00:00<?, ?B/s]
100% 500k/500k [00:00<00:00, 119MB/s]


In [6]:
# unzip all data
import zipfile
import os

basepath = '/content/competitions/11785-spring2021-hw1p2'

with os.scandir(basepath) as entries:
  for entry in entries:
    if entry.is_file():
      filename = entry.name
      if filename.endswith('.zip'):

        # check if file already unzipped
        raw_name = filename.replace(".zip", "")
        if os.path.exists(f'{basepath}/{raw_name}'):
          print(f"file {raw_name} already unzipped")
        else:
          print(f"unzipping {filename}")
          # unzip
          with zipfile.ZipFile('{}/{}'.format(basepath, entry.name), 'r') as zip_ref:
            zip_ref.extractall(basepath)


unzipping train.npy.zip
unzipping sample.csv.zip
unzipping dev_labels.npy.zip
unzipping test.npy.zip
unzipping dev.npy.zip


In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Preprocess data

In [ ]:
# Goal: generate .npy files for each PHONEME, inserting the frames that contain that phoneme label

In [ ]:
# 1. download kaggle data
# 2. dictionary of phoneme -> list of frames
# 3. store .npy per phoneme

In [ ]:
# to find frames for specific phoneme: stack complete frames, stack complete labels
# apply a group by operation

In [7]:
import numpy as np
import os
import torch

In [8]:
def stack_utterance_features(array_3d):
  """
  Stack all the utterance elements into one single array

  original input: [x,y,z]
  final input: [x*y, z]

  """
  return np.vstack(array_3d)


In [9]:
def stack_utterance_labels(array_3d):
  """
  Stack all the utterance elements into one single array

  original input: [x,y,z]
  final input: [x*y, z]

  """
  res = []
  for utterance in array_3d:
    for element in utterance:
      res.append(element)

  return np.array(res)


In [10]:
def save_numpy_to_file(filename, ndarray):
  with open(filename, 'wb') as f:
    np.save(f, ndarray)

In [11]:
def save_tensor_to_file(filename, ndarray):
  with open(filename, 'wb') as f:
    torch.save(ndarray, f)

Processs the files

In [46]:
basepath = '/content/competitions/11785-spring2021-hw1p2'
drivepath = '/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/DL_Group_Project/Dataset/Hw1p2_frames'


In [35]:
with os.scandir(basepath) as entries:
  for entry in entries:
    if entry.is_file():
      filename = entry.name

      if filename == "dev.npy":
        original_frames = np.load(f'{basepath}/{filename}', allow_pickle=True)
        print(f"original_frames.shape: {original_frames.shape}")
        output_frames = stack_utterance_features(array_3d=original_frames)
        print(f"output_frames.shape: {output_frames.shape}")

      if filename == "dev_labels.npy":
        original_labels = np.load(f'{basepath}/{filename}', allow_pickle=True)
        print(f"original_frames.shape: {original_labels.shape}")
        output_labels = stack_utterance_labels(array_3d=original_labels)
        print(f"output_labels.shape: {output_labels.shape}")

        """
        print(f"{filename} stacked size: {output.shape}")
        save_tensor_to_file(torch_processed_file, output)
        print(f"saved {torch_processed_file}")
        
        print("")
        """

original_frames.shape: (2200,)
output_frames.shape: (1559057, 40)
original_frames.shape: (2200,)
output_labels.shape: (1559057,)


In [42]:
# load phoneme.txt
phoneme_states_idx_to_str = {}
with open(f'{basepath}/phonemes.txt', 'r') as reader:
  for line in reader.readlines():
    parts = line.strip().split(" ")
    phoneme_states_idx_to_str[int(parts[1])] = parts[0]

# NOTE: we have 3 states for each phoneme (i.e: AA has AA0, AA1, AA2)
print(f"phoneme_states_idx_to_str: {phonemes_idx_to_str}")
print(f"{len(phonemes_idx_to_str)}")

phoneme_states_idx_to_str: {'0': 'SIL', '1': 'SPN', '2': 'AA0', '3': 'AA1', '4': 'AA2', '5': 'AE0', '6': 'AE1', '7': 'AE2', '8': 'AH0', '9': 'AH1', '10': 'AH2', '11': 'AO0', '12': 'AO1', '13': 'AO2', '14': 'AW0', '15': 'AW1', '16': 'AW2', '17': 'AY0', '18': 'AY1', '19': 'AY2', '20': 'B', '21': 'CH', '22': 'D', '23': 'DH', '24': 'EH0', '25': 'EH1', '26': 'EH2', '27': 'ER0', '28': 'ER1', '29': 'ER2', '30': 'EY0', '31': 'EY1', '32': 'EY2', '33': 'F', '34': 'G', '35': 'HH', '36': 'IH0', '37': 'IH1', '38': 'IH2', '39': 'IY0', '40': 'IY1', '41': 'IY2', '42': 'JH', '43': 'K', '44': 'L', '45': 'M', '46': 'N', '47': 'NG', '48': 'OW0', '49': 'OW1', '50': 'OW2', '51': 'OY0', '52': 'OY1', '53': 'OY2', '54': 'P', '55': 'R', '56': 'S', '57': 'SH', '58': 'T', '59': 'TH', '60': 'UH0', '61': 'UH1', '62': 'UH2', '63': 'UW0', '64': 'UW1', '65': 'UW2', '66': 'V', '67': 'W', '68': 'Y', '69': 'Z', '70': 'ZH'}
71


In [36]:
# find unique labels
unique_labels, counts = np.unique(output_labels, return_counts=True)
print(unique_labels.shape)
print(unique_labels)
print(counts)

(70,)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70.]
[264389   8662    434  20312    619    434  42353   1229  67410  25682
    512    271  24557    642     28  15226    484    537  36828   1958
  18182  10955  49011  24216    610  34034   1231  34364  11674    107
    135  29227   1591  30162  10898  25823  32047  26636   1202  28971
  30679    765   7631  37355  54902  38265  74650  15711   2320  22566
   1014   2992     74  27107  49626  83860  15202  75760   9879     45
   4760    157    945  17823    615  21821  28317   7323  42475    775]


In [40]:
def get_frames_for_phoneme_index(phoneme_states_idx_to_str, phoneme_idx):
  # group the frames by labels
  phoneme_frame_indexes = np.where(output_labels==phoneme_idx)[0]
  print(phoneme_frame_indexes.shape)
  phoneme_frames = output_frames[phoneme_frame_indexes]
  print(phoneme_frames.shape)

  # map phoneme_idx to phoneme
  phoneme = phoneme_states_idx_to_str[int(phoneme_idx)]

  return phoneme, phoneme_frames

In [47]:
for label in unique_labels:
  phoneme, phoneme_frames = get_frames_for_phoneme_index(phoneme_states_idx_to_str, label)
  # save
  save_numpy_to_file(filename=f"{drivepath}/{phoneme}.npy", ndarray=phoneme_frames)


(264389,)
(264389, 40)
(8662,)
(8662, 40)
(434,)
(434, 40)
(20312,)
(20312, 40)
(619,)
(619, 40)
(434,)
(434, 40)
(42353,)
(42353, 40)
(1229,)
(1229, 40)
(67410,)
(67410, 40)
(25682,)
(25682, 40)
(512,)
(512, 40)
(271,)
(271, 40)
(24557,)
(24557, 40)
(642,)
(642, 40)
(28,)
(28, 40)
(15226,)
(15226, 40)
(484,)
(484, 40)
(537,)
(537, 40)
(36828,)
(36828, 40)
(1958,)
(1958, 40)
(18182,)
(18182, 40)
(10955,)
(10955, 40)
(49011,)
(49011, 40)
(24216,)
(24216, 40)
(610,)
(610, 40)
(34034,)
(34034, 40)
(1231,)
(1231, 40)
(34364,)
(34364, 40)
(11674,)
(11674, 40)
(107,)
(107, 40)
(135,)
(135, 40)
(29227,)
(29227, 40)
(1591,)
(1591, 40)
(30162,)
(30162, 40)
(10898,)
(10898, 40)
(25823,)
(25823, 40)
(32047,)
(32047, 40)
(26636,)
(26636, 40)
(1202,)
(1202, 40)
(28971,)
(28971, 40)
(30679,)
(30679, 40)
(765,)
(765, 40)
(7631,)
(7631, 40)
(37355,)
(37355, 40)
(54902,)
(54902, 40)
(38265,)
(38265, 40)
(74650,)
(74650, 40)
(15711,)
(15711, 40)
(2320,)
(2320, 40)
(22566,)
(22566, 40)
(1014,)
(1014, 40)